In [27]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

In [45]:
pedido_post = pd.read_csv('compras.csv').drop(['Unnamed: 0'], axis=1)
pedido_post = pedido_post.T.reindex(['4','5']).T
pedido_post.columns = ['valorfinal', 'marca']
pedido_post

,valorfinal,marca
0,350.0,Nilson
1,124.95,Gatão
2,544.7,Marfrig
3,82.5,Danta
4,82.5,Danta
5,82.5,Danta
6,82.5,Danta
7,316.8,Cara Preta
8,240.0,Espetinhos
9,64.8,Dona Beth


In [47]:
# variáveis
two_months_ago = datetime.today() - timedelta(weeks=4)


In [46]:



# ## Pedido
# pedido
pedido = pd.DataFrame(pedido_post)
#pedido['valorfinal'] = pd.to_numeric([x.split()[1] for x in pedido['valorfinal']])
pedido = pd.pivot_table(pedido, values=['valorfinal'], index='marca', aggfunc='sum').reset_index()
pedido.columns = ['marca', 'total']
pedido

,marca,total
0,Cara Preta,316.80
1,Danta,330.00
2,Dona Beth,382.85
3,Espetinhos,240.00
4,Gatão,533.53
5,Marfrig,544.70
6,Nilson,350.00
7,Porcão,474.50
8,Saudali,180.00
9,Tirolez,297.00


In [48]:


# prazos
prazos = pd.read_csv('../prazos.csv')
prazos.columns = ['marca',1,2,3]
prazos['n_parcelas'] = 3 - prazos.T.isna().sum()
prazos = prazos.merge(pedido)
prazos

,marca,1,2,3,n_parcelas,total
0,Cara Preta,7.0,14.0,NaN,2,316.80
1,Danta,21.0,NaN,NaN,1,330.00
2,Dona Beth,14.0,NaN,NaN,1,382.85
3,Gatão,7.0,NaN,NaN,1,533.53
4,Marfrig,11.0,18.0,25.0,3,544.70
5,Porcão,7.0,14.0,NaN,2,474.50
6,Porcão,7.0,14.0,NaN,2,474.50
7,Nilson,14.0,NaN,NaN,1,350.00
8,Saudali,7.0,NaN,NaN,1,180.00
9,Tirolez,7.0,NaN,NaN,1,297.00


In [50]:


# agenda
agenda = prazos.copy()
agenda
for parcela in [1,2,3]:
    agenda[parcela] = [datetime.today() + timedelta(days=x) if x > 0 else None for x in prazos[parcela]]
agenda['valor'] = agenda['total']/agenda['n_parcelas']
agenda = agenda.melt(id_vars=['marca', 'valor'], value_vars=[1,2,3], value_name='data', var_name='parcela').dropna()
agenda['semana'] = [x.isocalendar()[1] for x in agenda['data']]
agenda


,marca,valor,parcela,data,semana
0,Cara Preta,158.400000,1,2022-06-06 18:25:19.081516,23
1,Danta,330.000000,1,2022-06-20 18:25:19.081516,25
2,Dona Beth,382.850000,1,2022-06-13 18:25:19.081516,24
3,Gatão,533.530000,1,2022-06-06 18:25:19.081516,23
4,Marfrig,181.566667,1,2022-06-10 18:25:19.081516,23
5,Porcão,237.250000,1,2022-06-06 18:25:19.081516,23
6,Porcão,237.250000,1,2022-06-06 18:25:19.081516,23
7,Nilson,350.000000,1,2022-06-13 18:25:19.081516,24
8,Saudali,180.000000,1,2022-06-06 18:25:19.081516,23
9,Tirolez,297.000000,1,2022-06-06 18:25:19.081516,23


In [55]:


# peso
peso = agenda.copy()
peso = pd.pivot_table(agenda, index=['data'], values='valor', aggfunc='sum').reset_index()
peso

,data,valor
0,2022-06-06 18:25:19.081516,1643.430000
1,2022-06-10 18:25:19.081516,181.566667
2,2022-06-13 18:25:19.081516,1365.750000
3,2022-06-17 18:25:19.081516,181.566667
4,2022-06-20 18:25:19.081516,330.000000
5,2022-06-24 18:25:19.082515,181.566667


In [56]:



# ## Situação
# contas
vendas = list()
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'dataInicial': two_months_ago.strftime('%d/%m/%Y'),
    'dataFinal': datetime.today().strftime('%d/%m/%Y'),
    'pagina': 1
}
while True:
    try:

        url = 'https://api.tiny.com.br/api2/pedidos.pesquisa.php'
        vendas += [x['pedido'] for x in requests.post(url, data).json()['retorno']['pedidos']]
        data['pagina'] += 1
    except:
        break
vendas = pd.DataFrame(vendas)


In [58]:


# faturamento diário
faturamento_estimado = vendas['valor'].sum()/30
faturamento_estimado

3163.139333333333

In [60]:


# contas
contas = list()
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'data_ini_vencimento': datetime.today().strftime('%d/%m/%Y'),
    'data_fim_vencimento': agenda['data'].max().strftime('%d/%m/%Y'),
    'pagina': 1
}
while True:
    try:
        url = 'https://api.tiny.com.br/api2/contas.pagar.pesquisa.php'
        contas += [x['conta'] for x in requests.post(url, data).json()['retorno']['contas']]
        data['pagina'] += 1
    except:
        break
contas = pd.DataFrame(contas)
contas = contas.T.reindex(['data_vencimento', 'valor']).T
contas.columns = ['data', 'valor']
contas['data'] = [datetime.strptime(x, '%d/%m/%Y') for x in contas['data']]
contas


,data,valor
0,2022-05-31,8877.90
1,2022-06-01,325.92
2,2022-06-01,1325.00
3,2022-06-01,407.00
4,2022-06-01,393.00
5,2022-06-01,393.88
6,2022-06-01,1000.00
7,2022-06-02,1078.89
8,2022-06-03,191.85
9,2022-06-05,205.02


In [61]:


# saldo
saldo = pd.DataFrame(contas['data'])
saldo['valor'] = np.repeat(faturamento_estimado, len(contas)) - pd.to_numeric(contas['valor'])



In [93]:


# ## Divisão por semanas
# saldo
saldo_semanal = saldo.copy()
saldo_semanal['semana'] = [x.isocalendar()[1] for x in saldo['data']]
saldo_semanal = pd.pivot_table(saldo_semanal, index='semana', values='valor', aggfunc='sum')
saldo_semanal['valor'] = pd.to_numeric(saldo_semanal['valor'])
saldo_semanal


,valor
semana,
22,11596.072667
23,4310.118000
25,7396.088000


In [92]:

# compras
peso_semanal = peso.copy()
peso_semanal['semana'] = [x.isocalendar()[1] for x in peso['data']]
peso_semanal = pd.pivot_table(peso_semanal, index='semana', values='valor', aggfunc='sum')
peso_semanal['valor'] = pd.to_numeric(peso_semanal['valor'])
peso_semanal


,valor
semana,
23,1824.996667
24,1547.316667
25,511.566667


In [100]:

# ## resultado
# analise
def rweek(semana):
    return datetime.fromisocalendar(datetime.today().year, semana, 1), datetime.fromisocalendar(datetime.today().year, semana, 7)
analise = peso_semanal.join(saldo_semanal, lsuffix='_compras',rsuffix='_saldo').fillna(faturamento_estimado*7)


analise['subtracao'] = pd.to_numeric(analise['valor_saldo']) - pd.to_numeric(analise['valor_compras'])
analise['intervalo'] = [str(rweek(x)[0].strftime('%d/%m') + ' à ' + rweek(x)[1].strftime('%d/%m')) for x in analise.index]
analise['aprovado'] = [x > 0 for x in analise['subtracao']]
analise.reset_index(inplace=True)
analise = analise.T.reindex(['aprovado', 'intervalo', 'subtracao', 'valor_compras', 'valor_saldo', 'semana']).T
for column in ['subtracao', 'valor_saldo', 'valor_compras']:
    analise[column] = pd.to_numeric(analise[column]).round(2)
analise.drop(['semana'], axis=1)



,aprovado,intervalo,subtracao,valor_compras,valor_saldo
0,True,06/06 à 12/06,2485.12,1825.00,4310.12
1,True,13/06 à 19/06,20594.66,1547.32,22141.98
2,True,20/06 à 26/06,6884.52,511.57,7396.09


In [67]:

# negadas
negadas = analise[analise['aprovado'] == False]
negadas = negadas.merge(agenda)
negadas = negadas.T.reindex(['marca', 'data', 'valor']).T
negadas


,marca,data,valor
